In [3]:
import pandas as pd
import numpy as np

In [17]:
df_test = pd.read_csv('test.csv')

In [14]:
def explode_position_scalable(df_in, col_expl, pipe='|'):
    """Explode column col_expl of array type into multiple rows."""
    
    n = 60000  #chunk row size
    list_df = [df_in[i:i+n] for i in range(0,df_in.shape[0],n)]
    # Handle the first alone to have the correct headers
    df_x = explode_position(list_df[0], col_expl, flag_conversion=False, pipe_type=pipe)
    df_x = df_x.rename(columns={col_expl:'item_id'})
    df_x.to_csv('exploded.csv', index=False)
    list_df.pop(0)
    for df_x in list_df:
        df_x = explode_position(df_x, col_expl, flag_conversion=False, pipe_type=pipe)
        df_x = df_x.rename(columns={col_expl:'item_id'})
        df_x.to_csv('exploded.csv', mode='a', header=False, index=False)

    df_out = pd.read_csv('exploded.csv')

    os.remove('exploded.csv')
    return df_out
  
def explode_position(df_in, col_expl, flag_conversion = True, pipe_type='|'):
    """Explode column col_expl of array type into multiple rows."""

    df = df_in.copy()
    if(pipe_type== '|'):
        df.loc[:, col_expl] = df[col_expl].apply(string_to_array)
    elif(pipe_type==' '):
        df.loc[:, col_expl] = df[col_expl].apply(space_to_array)
    df.loc[:, 'position'] = df[col_expl].apply(get_position)
    df_out = pd.DataFrame(
        {col: np.repeat(df[col].values,
                        df[col_expl].str.len())
         for col in df.columns.drop(col_expl)}
    )
    df_out.loc[:, col_expl] = np.concatenate(df[col_expl].values)
    df_out.loc[:, 'position'] = np.concatenate(df['position'].values)
    if(flag_conversion):
        df_out.loc[:, col_expl] = df_out[col_expl].apply(int)

    return df_out
  
def string_to_array(s):
    """Convert pipe separated string to array."""

    if isinstance(s, str):
        out = s.split("|")
    elif math.isnan(s):
        out = []
    else:
        raise ValueError("Value must be either string of nan")
    return out
  
def get_position(l):
    pos = []
    for i in range(0,len(l)):
        pos.append(i)
    return pos

def create_recent_index(df_orig, grouped=False):
    # distinct_hotel = group.reference.drop_duplicates().values
    # dict = {}
    # counter = 0
    # for x in distinct_hotel:
    #     dict[x] = counter
    #     counter += 1
    df_list_int = df_orig.groupby('session_id').apply(lambda x: get_list_session_interactions(x)).reset_index(name='all_interactions')
    df_list_int = df_list_int[['session_id', 'all_interactions']]
    if(grouped):
        return df_list_int
    df_orig = (df_orig.merge(df_list_int, left_on=['session_id'], right_on=['session_id'], how="left"))
    #del df_orig['all_interactions']
    return df_orig
  
def get_list_session_interactions(group):
    group = group[group['action_type'] != 'clickout item']
    group.loc[:,'reference'] = group['reference'].apply(str)
    list_values = group.reference.drop_duplicates()
    joined = " ".join(list_values)
    return joined

def get_submission_target(df):
    """Identify target rows with missing click outs."""

    mask = df["reference"].isnull() & (df["action_type"] == "clickout item")
    df_out = df[mask]

    return df_out

In [18]:
mask = (df_test["action_type"] == "clickout item") | (df_test["action_type"] == "interaction item rating") | (df_test["action_type"] == "interaction item image") | (df_test["action_type"] == "interaction item deals")
df_test = df_test[mask]

In [20]:
df_recent = create_recent_index(df_test)
df_recent

,user_id,session_id,timestamp,step,action_type,reference,platform,city,device,current_filters,impressions,prices,all_interactions
0,03P4VFKK12UO,325fafb5fa450,1541107435,17,interaction item image,65685,US,"Bakersfield, USA",desktop,NaN,NaN,NaN,65685
1,03P4VFKK12UO,325fafb5fa450,1541107435,18,interaction item image,65685,US,"Bakersfield, USA",desktop,NaN,NaN,NaN,65685
2,03P4VFKK12UO,325fafb5fa450,1541107457,19,interaction item image,65685,US,"Bakersfield, USA",desktop,NaN,NaN,NaN,65685
3,03P4VFKK12UO,325fafb5fa450,1541107457,20,interaction item image,65685,US,"Bakersfield, USA",desktop,NaN,NaN,NaN,65685
4,03P4VFKK12UO,325fafb5fa450,1541107467,21,interaction item image,65685,US,"Bakersfield, USA",desktop,NaN,NaN,NaN,65685
5,03P4VFKK12UO,325fafb5fa450,1541107467,22,interaction item image,65685,US,"Bakersfield, USA",desktop,NaN,NaN,NaN,65685
6,03P4VFKK12UO,325fafb5fa450,1541107467,23,interaction item image,65685,US,"Bakersfield, USA",desktop,NaN,NaN,NaN,65685
7,03P4VFKK12UO,325fafb5fa450,1541107467,24,interaction item image,65685,US,"Bakersfield, USA",desktop,NaN,NaN,NaN,65685
8,03P4VFKK12UO,325fafb5fa450,1541107467,25,interaction item image,65685,US,"Bakersfield, USA",desktop,NaN,NaN,NaN,65685
9,03P4VFKK12UO,325fafb5fa450,1541107467,26,interaction item image,65685,US,"Bakersfield, USA",desktop,NaN,NaN,NaN,65685
